<a href="https://colab.research.google.com/github/UDEA-Esp-Analitica-y-Ciencia-de-Datos/EACD-04-MACHINE-LEARNING-1/blob/master/15-%5BTALLER%5D_Modelos_basados_en_arboles_importancia_de_variables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Recuerda que una vez abierto, Da clic en "Copiar en Drive", de lo contrario no podras alamancenar tu progreso**

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/UDEA-Esp-Analitica-y-Ciencia-de-Datos/EACD-04-MACHINE-LEARNING-1/master/init.py
import init; init.init(force_download=False); 

# Parte 1.

**Bag of words + Bagging**


## Contextualización del problema


Usaremos el dataset Twitter US Airline Sentiment para reesolver un problema de análisis de sentimiento en texto. En el repositorio de Kaggle se encuentra más información en el siguiente [link](https://www.kaggle.com/crowdflower/twitter-airline-sentiment)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('local/data/Tweets.csv')
# Keeping only the neccessary columns
data = data[['text','airline_sentiment']]

Como es un problema de procesamiento de texto debemos hacer algunas tareas báscias de preprocesamiento:

In [ ]:
import re
#Remove neutral class
data = data[data.airline_sentiment != "neutral"]

#text normalization
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x:re.sub('@[^\s]+','',x)))#remove the name of the airline
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
print(f"Número de muestras positivas {np.sum(data['airline_sentiment'].values == 'positive')}")
print(f"Número de muestras negatias {np.sum(data['airline_sentiment'].values == 'negative')}")

In [ ]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

In [ ]:
data[:10]

La forma más básica de representar textos para problemas de clasificación es usar la estrategia Bag of Words [link](https://en.wikipedia.org/wiki/Bag-of-words_model#:~:text=The%20bag%2Dof%2Dwords%20model,word%20order%20but%20keeping%20multiplicity.)

# Bag of words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#define vectorizer parameters
count_vectorizer = CountVectorizer(max_features=2000, stop_words='english')

%time count_matrix = count_vectorizer.fit_transform(data['text']) #fit the vectorizer to synopses

print(count_matrix.shape)
count_matrix.toarray()

## Ejercicio 1.1: Clasificación usando estrategias de Bagging

Realice la partición de los datos entre Entrenamiento y Test. Recuerde que Test es el subconjunto que dejará pára medir el desempeño del sistema después del entrenamiento y de la selección de los hiperparámetros. 

Teniendo en cuenta la distribución de clases, seleccione correctamente la estrategia de particionamiento.

In [ ]:
#Ejercicio de código
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

#Complete el código
X_train, X_test, y_train, y_test = train_test_split(..., test_size=0.2, random_state=42, stratify=...)

st = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

Diseñe un experimento para ajustar los hiper-parámetros de un modelo Random Forest. Ajuste el número de árboles, la profundida de cada árbol y el número de variables a analizar por nodo. Recuerde que como el problema es desbalanceado debe usar el parámetro **class_weight='balanced_subsample'** para la definición del modelo.

In [ ]:
#Ejercicio de código
from sklearn.ensemble import RandomForestClassifier

clf_b = ... #use random_state=0

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'n_estimators':[20,40,60,80,100,120], 'max_depth':[2,4,6,8], 'max_features':[10,20,30,40,50]}

clf = GridSearchCV(estimator=..., param_grid=parameters, cv=st, scoring='balanced_accuracy')

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
clf.best_estimator_

Evalúe en las muestras de test. Estime: Accuracy, Accuracy Balanceado y F1. Grafique la matriz de confusión normalizada. Para eso ejecute la siguiente celda. Analice el código usado.

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder().fit(np.unique(data['airline_sentiment']))

Yest = clf.predict(X_test)

print(f"Accuracy = {accuracy_score(y_test,Yest)}")
print(f"Balanced Accuracy = {balanced_accuracy_score(y_test,Yest)}")

#Las métricas F1, precision and recall requieren que se establezca la convención de cuál es la clase positiva (1)
print(f"F1 = {f1_score(le.transform(y_test),le.transform(Yest))}")

disp = plot_confusion_matrix(clf, X_test, y_test, display_labels=np.unique(data['airline_sentiment']),
                             cmap=plt.cm.Blues, 
                             normalize='true')
disp.ax_.set_title('MC normalizada')
plt.show()

## Ejercicio 1.2

Diseñe un experimento para ajustar los hiper-parámetros de un modelo de Bagging de árboles de decisión. Ajuste el número de árboles y la profundida de cada árbol.

Tenga en cuenta que el GridSearch no puede modificar la profundida del árbol de decisión porque sólo puede acceder a los parámetros del estimador, que en este caso es un modelo de Bagging, la profundidad hace parte de los párametros del estimador base y no son accesibles por el GridSearch.

In [ ]:
#Ejercicio de código
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

max_depth = [2,4,6,8]
parameters = {'n_estimators':[20,40,60,80,100]}
clf2 = []
for i in max_depth:
    #Complete el código
    clf_b = ...
    clf_bagging = BaggingClassifier(base_estimator=clf_b, random_state=0)
    clf2.append(GridSearchCV(estimator=clf_bagging, param_grid=parameters, cv=st, scoring='balanced_accuracy').fit(X_train,y_train))

best_score = 0
for i in range(len(max_depth)):
    if clf2[i].best_score_ >= best_score:
        best_score = clf2[i].best_score_
        ind = i
clf2 = clf2[ind]

In [ ]:
clf2.best_estimator_

Evalúe en las muestras de test. Estime Accuracy, Accuracy Balanceado y F1. Grafique la matriz de confusión.

In [ ]:
Yest = clf2.predict(X_test)

print(f"Accuracy = {accuracy_score(y_test,Yest)}")
print(f"Balanced Accuracy = {balanced_accuracy_score(y_test,Yest)}")

#Las métricas F1, precision and recall requieren que se establezca la convención de cuál es la clase positiva (1)
print(f"F1 = {f1_score(le.transform(y_test),le.transform(Yest))}")

disp = plot_confusion_matrix(clf2, X_test, y_test, display_labels=np.unique(data['airline_sentiment']),
                             cmap=plt.cm.Blues, 
                             normalize='true')
disp.ax_.set_title('MC normalizada')
plt.show()

## Ejercicio 1.3: Importancia de variables

Grafique la importancia de las variables de acuerdo con el mejor de los dos comités de máquina evaluados. Determine cuántas variables tienen importancia 0.

In [ ]:
#Ejercicio de código
plt.bar(...

¿Cuáles son las 10 palabras más determinantes para definir el sentimiento de los tweets? Incluya el código necesario para poder responder la pregunta.

**Ayuda**: count_vectorizer.vocabulary_ contiene el diccionario de palabras usado.

In [ ]:
#Ejercicio de código



Cree nueva versiones para los conjuntos Xtrain y Xtest, eliminando las variables que según el análisis anterior tienen importancia cero. Realice nuevamente el proceso de entrenamiento y validación.

In [ ]:
#Ejercicio de código
X_train2 = X_train[:,...]
X_test2 = X_test[:,...]


parameters = ...
clf_b = ...
clf3 = GridSearchCV(estimator=clf_b, param_grid=parameters, cv=st, scoring=...)
clf3.fit(X_train2,y_train)

In [ ]:
clf3.best_estimator_.n_estimators

Evalúe en las muestras de test. Estime Accuracy, Accuracy Balanceado y F1. Grafique la matriz de confusión. ¿Cambió el resultado?

In [ ]:
Yest = clf3.predict(X_test2)

print(f"Accuracy = {accuracy_score(y_test,Yest)}")
print(f"Balanced Accuracy = {balanced_accuracy_score(y_test,Yest)}")

#Las métricas F1, precision and recall requieren que se establezca la convención de cuál es la clase positiva (1)
print(f"F1 = {f1_score(le.transform(y_test),le.transform(Yest))}")

disp = plot_confusion_matrix(clf3, X_test2, y_test, display_labels=np.unique(data['airline_sentiment']),
                             cmap=plt.cm.Blues, 
                             normalize='true')
disp.ax_.set_title('MC normalizada')
plt.show()

In [ ]:
#@title Pregunta Abierta
#@markdown  ¿Cambió el resultado del modelo de manera significativa al reducir el número de variables?
respuesta_1 = "" #@param {type:"string"}

# Parte 2.

**tf-idf + Boosting**

En este caso vamos a usar una estrategia un poco más robusta para la caracterización de los textos, se llama tf-idf.

## tf-idf representation

This stands for term frequency and inverse document frequency. The tf-idf weighting scheme assigns to term $t$ a weight in document $d$ given by

$$
\mbox{tf-idf}_{t,d} = \mbox{tf}_{t,d} \times \mbox{idf}_t.$$

**Term Frequency (tf)**: gives us the frequency of the word in each document in the corpus. It is the ratio of number of times the word appears in a document compared to the total number of words in that document. It increases as the number of occurrences of that word within the document increases. Each document has its own tf.

$$\mbox{tf}_{t,d} = \frac{n_{t,d}}{\sum_k n_{k,d}} $$

**Inverse Data Frequency (idf)**: used to calculate the weight of rare words across all documents in the corpus. The words that occur rarely in the corpus have a high IDF score. It is given by the equation below.

$$\mbox{idf}_{t} = \log\left(\frac{N}{df_t}\right) + 1; \; df_t = \text{number of documents contaning } t $$

In other words, $\mbox{tf-idf}_{t,d}$ assigns to term $t$ a weight in document $d$ that is

- highest when $t$ occurs many times within a small number of documents (thus lending high discriminating power to those documents);
- lower when the term occurs fewer times in a document, or occurs in many documents (thus offering a less pronounced relevance signal);

- lowest when the term occurs in virtually all documents.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_features=2000,stop_words='english',
                                 use_idf=True) #ngram_range=(1,3)

%time tfidf_matrix = tfidf_vectorizer.fit_transform(data['text']) #fit the vectorizer to synopses

print(tfidf_matrix.shape)
print(tfidf_matrix)

**Nota**: Solo de carácter informativo. El proceso de construcción de la matriz tf-ids se puede realizar de manera alternativa usando la clase TfidfTransformer a partir de la matrix de conteos construida en el punto 1:

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
tfidf_matrix2 = tfidf_transformer.fit_transform(count_matrix) #fit the vectorizer to synopses

print(tfidf_matrix2.shape)
print(tfidf_matrix2)

## Ejercicio 2.1: Clasificación usando estrategias de Boosting

Realice la partición de los datos entre Entrenamiento y Test. Recuerde que Test es el subconjunto que dejará pára medir el desempeño del sistema después del entrenamiento y de la selección de los hiperparámetros. 

Teniendo en cuenta la distribución de clases, seleccione correctamente la estrategia de particionamiento.

In [ ]:
#Complete el código
X_train, X_test, y_train, y_test = train_test_split(...

Diseñe un experimento para ajustar los hiper-parámetros de un modelo Gradient Boosting Tree. Ajuste el número de árboles, la profundida de cada árbol y el número de variables a analizar por nodo.

**Note** que en este caso el modelo GBT no cuenta con un parámetro para cambiar el peso que cada clase tiene en el entrenamiento.

In [ ]:
#Complete el código
from sklearn.ensemble import GradientBoostingClassifier

clf_b = GradientBoostingClassifier(random_state=0)

parameters = ...

clf = GridSearchCV(estimator=clf_b, param_grid=parameters, cv=st, scoring=...)

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
clf.best_estimator_.n_estimators

Evalúe en las muestras de test. Estime Accuracy, Accuracy Balanceado y F1. Grafique la matriz de confusión.

In [ ]:
Yest = clf.predict(X_test)

print(f"Accuracy = {accuracy_score(y_test,Yest)}")
print(f"Balanced Accuracy = {balanced_accuracy_score(y_test,Yest)}")

#Las métricas F1, precision and recall requieren que se establezca la convención de cuál es la clase positiva (1)
print(f"F1 = {f1_score(le.transform(y_test),le.transform(Yest))}")

disp = plot_confusion_matrix(clf, X_test, y_test, display_labels=np.unique(data['airline_sentiment']),
                             cmap=plt.cm.Blues, 
                             normalize='true')
disp.ax_.set_title('MC normalizada')
plt.show()

## Ejercicio 2.2: 

Modifique el experimento anterior para ajustar los hiper-parámetros de un modelo **Stochastic** Gradient Boosting Tree. Ajuste el número de árboles, la profundida de cada árbol y el número de variables a analizar por nodo. Use un parámetro subsample = 0.8.

In [ ]:
#Complete el código
clf_b = GradientBoostingClassifier(...

parameters = ...

clf2 = GridSearchCV(estimator=clf_b, param_grid=parameters, cv=st, scoring=...)
clf2.fit(X_train,y_train)

In [ ]:
clf2.best_estimator_

Evalúe en las muestras de test. Estime: Accuracy, Accuracy Balanceado y F1. Grafique la matriz de confusión normalizada.

In [ ]:
Yest = clf2.predict(X_test)

print(f"Accuracy = {accuracy_score(y_test,Yest)}")
print(f"Balanced Accuracy = {balanced_accuracy_score(y_test,Yest)}")

#Las métricas F1, precision and recall requieren que se establezca la convención de cuál es la clase positiva (1)
print(f"F1 = {f1_score(le.transform(y_test),le.transform(Yest))}")

disp = plot_confusion_matrix(clf2, X_test, y_test, display_labels=np.unique(data['airline_sentiment']),
                             cmap=plt.cm.Blues, 
                             normalize='true')
disp.ax_.set_title('MC normalizada')
plt.show()

## Ejercicio 2.3

Determine las 10 palabras más importantes para la clasificación de acuerdo con el mejor modelo de la parte 2.

**Ayuda**: tfidf_vectorizer.vocabulary_ contiene el diccionario de palabras usado.

In [ ]:
#Complete el código



## *Ejercicio 3: XGBoost

Se recomienda revisar la documentación oficial [link](https://xgboost.readthedocs.io/en/latest/index.html). XGBoost tiene una sintaxis diferente a sklearn pero incluye un wrapper que permite usarlo igual que un modelo estándar de sklearn y por lo tanto permite usar GridSearch para realizar el ajuste de los hiperparámetros.

In [ ]:
import xgboost as xgb
from sklearn.utils.class_weight import compute_class_weight

XGBoost tiene muchos más ṕarámetros que el GBT estándar. Además del número de estimadores, la profundidad del árbol y la tasa de aprendizaje (también en el GBT), XGBoost contiene parámetros de regularización $l_1$ y $l_2$ y a través del parámetro scale_pos_weight sepuede modificar el peso de la clase positiva en relación con la negativa.

Analice la manera en que está ajustado ese parámetro en el siguiente código:

In [ ]:
class_weight = compute_class_weight('balanced',le.classes_,y_train)

In [ ]:
class_weight

In [ ]:
param = {'n_estimators':[20,40,60,80,100,120,140],'max_depth': [2,4,6,8], 'learning_rate':[1,0.5,0.1], 'reg_lambda':[0.1,0.2]}
clf_b = xgb.XGBClassifier(scale_pos_weight=class_weight[1]/class_weight[0],random_state = 0, objective='binary:logistic')

In [ ]:
clf3 = GridSearchCV(estimator=clf_b, param_grid=param, cv=st, scoring='balanced_accuracy')
clf3.fit(X_train,y_train)

In [ ]:
clf3.best_estimator_

In [ ]:
Yest = clf3.predict(X_test)

print(f"Accuracy = {accuracy_score(y_test,Yest)}")
print(f"Balanced Accuracy = {balanced_accuracy_score(y_test,Yest)}")

#Las métricas F1, precision and recall requieren que se establezca la convención de cuál es la clase positiva (1)
print(f"F1 = {f1_score(le.transform(y_test),le.transform(Yest))}")

disp = plot_confusion_matrix(clf3, X_test, y_test, display_labels=np.unique(data['airline_sentiment']),
                             cmap=plt.cm.Blues, 
                             normalize='true')
disp.ax_.set_title('MC normalizada')
plt.show()

Igual que para los casos anteriores determine el número de características con importancia diferente de cero, y las 10 palabras más importantes según el modelo.

In [ ]:
#Complete el código

